In [1]:
from baseline_main import main as baseline

In [2]:
from generation import gpt3 as gpt3_gen

In [3]:
import openai

In [4]:
question = {
    "question_id": "20220610_0_nota", 
    "question_date": "2022/06/10", 
    "question_source": "CNN", 
    "question_url": "https://edition.cnn.com/interactive/2022/06/us/cnn-5-things-news-quiz-june-9-sec/", 
    "question_sentence": "According to a recent ranking, which is the world’s most expensive city?", 
    "choices": ["New York", "Tokyo", "London", "None of the above"], 
    "answer": ["3"], 
    "evidence": "For the third year in a row, <a href=\"https://www.cnn.com/travel/article/world-most-expensive-cities-2022-intl-hnk/index.html\">Hong Kong</a> received the dubious title of “priciest city in the world,” according to an analysis this week by global mobility company ECA International. The company calculates the list based on several factors, including the average price of groceries, rent, utilities, public transit and the strength of the local currency."
}

In [5]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = api_key)

In [6]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
gpt3_gen.gpt3_question(question, tokenizer=tokenizer)

/home/yohan/work/MultiModal_RAG/realtime_qa/scripts/generation/gpt3.py:75: RuntimeWarning: Mean of empty slice.
  score = lprobs.mean()
/home/yohan/anaconda3/envs/multimodal_rag/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(['0'], 'nan')

In [10]:
# gpt3_gen.gpt3_questions step by step 
from utils.tools import add_today
retrived_text = None
model = 'gpt-3.5-turbo-instruct'
rm_date_q = False
tokeninzer = tokenizer

sentence = question["question_sentence"]
if not rm_date_q:
    sentence = add_today(sentence, question["question_date"])
prompt = "Question: " + sentence
choices = question["choices"]
prompt += "\n"


In [12]:
prompt, choices

('Question: Today is June 10, 2022. According to a recent ranking, which is the world’s most expensive city?\n',
 ['New York', 'Tokyo', 'London', 'None of the above'])

In [92]:
import string
import numpy as np
outputs = []
scores = []
lprobs_list = []
for alphabet, choice in zip(string.ascii_uppercase, choices):
    if "None of" in choice:
        ans = "No answer is in ({})".format(",".join(choices[:-1]))
    else:
        ans = "Assumed answer: {}) {}".format(alphabet, choice)
    

    #ans_len = len(tokenizer(ans)['input_ids']) - 1 # 요한님 코드
    ans_len = len(tokenizer.encode(ans)) - 1
    query = prompt + "\n" + ans
    if "None of" not in choice:
        query = query + "\n" + "Is assumed answer is right?"
    print(query)
    output = client.completions.create(
                            model = model,
                            prompt = query,
                            max_tokens = 50,
                            logprobs = 10,
                            # echo = True,
                            temperature = 0.1,
                            )
    # print(query, output.choices[0])
    #lprobs = np.array(output["choices"][0]["logprobs"]["token_logprobs"][1:])
    # assert output["choices"][0]["logprobs"]["tokens"][-ans_len-2] == "Answer"
    # assert output["choices"][0]["logprobs"]["tokens"][-ans_len-1] == ":"
    # lprobs = np.array(output["choices"][0]["logprobs"]["token_logprobs"][-ans_len:])
    outputs.append(output)
    lprobs = np.array(output.choices[0].logprobs.token_logprobs[-ans_len:])
    lprobs_list.append(lprobs)
    score = lprobs.mean()
    scores.append(score)
scores = np.array(scores)

Question: Today is June 10, 2022. According to a recent ranking, which is the world’s most expensive city?

Assumed answer: A) New York
Is assumed answer is right?
Question: Today is June 10, 2022. According to a recent ranking, which is the world’s most expensive city?

Assumed answer: B) Tokyo
Is assumed answer is right?
Question: Today is June 10, 2022. According to a recent ranking, which is the world’s most expensive city?

Assumed answer: C) London
Is assumed answer is right?
Question: Today is June 10, 2022. According to a recent ranking, which is the world’s most expensive city?

No answer is in (New York,Tokyo,London)


In [86]:
query

'Question: Today is June 10, 2022. According to a recent ranking, which is the world’s most expensive city?\n\nAssumed answer: D) None of the above\nIs assumed answer is right? Please answer yes or no.'

In [93]:
outputs, scores, lprobs_list

([Completion(id='cmpl-A4LqYYSpn3bTNQYvIEybhNbfzLkYX', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=Logprobs(text_offset=[163, 165, 167, 168, 172, 180, 187, 190, 194, 206, 212, 213, 217, 222, 232, 237, 241, 248, 253, 258, 262, 266, 270, 273, 277, 282, 285, 288, 292, 295, 299, 304, 305, 318, 319, 323, 331, 335, 340, 350, 353, 357, 364, 368, 377, 382, 383, 386, 389, 394], token_logprobs=[-0.46022198, -0.29617608, -0.0014285711, -0.20168114, -0.3661154, -2.8206474e-05, -0.07019829, -0.068431444, -0.12839827, -0.19668593, -0.5987323, -0.22217828, -1.0792534, -0.0065858127, -0.0011702254, -0.42165262, -0.0698107, -0.16125837, -2.057744e-05, -0.31744444, -0.8217548, -0.49686095, -0.8307071, -0.8486419, -0.00019626896, -0.44097278, -0.24727246, -0.009598239, -0.9921706, -0.63815206, -0.09862542, -0.1870891, -1.1313975, -1.11603495e-05, -0.80724484, -0.75506574, -0.4319724, -1.0132167, -0.020985823, -2.9352968e-06, -0.117144115, -0.18781465, -0.4584831, -0.6440892, -0.001

In [94]:
answer = scores.argmax()

In [95]:
answer

3

In [81]:
output = client.completions.create(
                        model = model,
                        prompt = "'Question: Today is June 10, 2022. According to a recent ranking, which is the world’s most expensive city? ",
                        max_tokens=50,
                        logprobs = 10,
                        # echo = True,
                        temperature = 0.05,
                        )
output

Completion(id='cmpl-A4Ldz0dMbuUhRrgE9yOeT9FbveIOA', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=Logprobs(text_offset=[108, 110, 116, 117, 120, 123, 128, 129, 131, 132, 133, 136, 137, 138, 142, 148, 150, 155, 165, 170, 173, 183, 190, 191, 203], token_logprobs=[-0.04529266, -0.079203494, -0.00038598618, -0.13697653, -0.0012314192, -0.15007427, -8.947716e-05, -0.08231551, -0.00067514315, -0.00033320097, -2.8444882e-05, -0.00015812746, -0.000723395, -0.27500144, -0.036534578, -0.091364674, -6.420598e-05, -3.166338e-05, -1.8550976e-05, -0.2695399, -1.2498171, -1.4580019, -0.00024323154, -0.0014956956, -0.5937745], tokens=['\n\n', 'Answer', ':', ' As', ' of', ' June', ' ', '10', ',', ' ', '202', '2', ',', ' the', ' world', "'s", ' most', ' expensive', ' city', ' is', ' currently', ' Zurich', ',', ' Switzerland', '.'], top_logprobs=[{'\n\n': -0.04529266, '\n': -3.4295528, "'\n\n": -5.161334, '\n\n\n': -6.222995, ' \n\n': -6.938819, "'\n": -7.5990286, 'Answer': -7.6569643

In [1]:
import os
print(os.environ.get("TRANSFORMERS_CACHE"))
print(os.environ.get("HF_HOME"))
print(os.environ.get("HF_DATASET_CACHE"))

None
None
None
